In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from helper_func import *

### Load Data

In [ ]:
#read cleaned data
response_pd = pd.read_csv('data/Responses/subset_data.csv')
response_gpd = gpd.GeoDataFrame(response_pd)

In [ ]:
# read london graphml File or download if not on disk
london_network = load_london_network()

### Reproject response data to BNG

In [ ]:
response_gpd['dispatched_location'] = [Point(response['lon_activation'], response['lat_activation']) for i, response in response_gpd.iterrows()]
response_gpd.geometry = response_gpd['dispatched_location']
response_gpd.crs = {'init': 'epsg:4326'}
response_gpd = response_gpd.to_crs({'init':'epsg:27700'})
response_gpd['dispatched_location'] = response_gpd.geometry

response_gpd['incident_location'] = [Point(response['lon_incident'], response['lat_incident']) for i, response in response_gpd.iterrows()]
response_gpd.geometry = response_gpd['incident_location']
response_gpd.crs = {'init': 'epsg:4326'}
response_gpd = response_gpd.to_crs({'init':'epsg:27700'})
response_gpd['incident_location'] = response_gpd.geometry

In [ ]:
response_pd.drop_duplicates(['lon_activation', 'lat_activation'], keep='first').to_csv('unique_activations')

## Save graph nodes to database

In [ ]:
schema = 'public'
table_name = 'nodes'
host = 'localhost'
database = 'postgres'
user = 'postgres'
password = 'admin'

graph_nodes_to_db(london_network, schema, table_name, host, database, user, password)

### Get shortest path between incident and dispatch

In [ ]:
# Creating SQLAlchemy's engine to use
DSN = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(
    user = user,
    password = password, 
    host = host,
    database = database, 
    port = 5432
    )

#create threaded connection pool to database to make concurrent calls
tcp = ThreadedConnectionPool(1,12,DSN)

In [ ]:
## find what is already on the database
conn = tcp.getconn()
c = conn.cursor()

#checks what activationid are already in shortest_path, is table is not in databse
#will throw an error
sql = "SELECT activationid FROM shortest_path"

c.execute(sql)
result = c.fetchall()
tcp.putconn(conn, close=True)

In [ ]:
#create list of id's already saved in shortest_path
result_id=[]
for res in result:
    result_id.append(res[0])

In [ ]:
#create a subset of all points not in table
temp = response_gpd[~response_gpd.activationid.isin(result_id)]

In [ ]:
#terate through each job and find shortest path, and save to database
cnt = 0
start_time = time.time()
for idx, row in temp.iterrows():
    if cnt%100 == 0:
        print('{}: {:,.2f} seconds'.format(cnt, time.time()-start_time))
    with ThreadPoolExecutor(max_workers=12) as pool:
        pool.submit(get_shortest_path,  row, tcp, table_name, london_network)
    cnt +=1